In [76]:
import requests
import time
url = "https://treeherder.mozilla.org/api/performance/alertsummary/"
alertsummaries = []
i = 0
while True:
    i += 1
    print(f"{i}. GET {url}", end="\r")
    r = requests.get(url, headers={'User-Agent': 'basic'})
    j = r.json()
    alertsummaries += j['results']
    url = j['next']
    if url is None:
        break
    time.sleep(0.5)

In [2]:
%cd ..
%pwd

/Users/markus/Documents/regression-prediction


'/Users/markus/Documents/regression-prediction'

In [38]:
from src.utils import *
from src.labeling import *

In [77]:
import datetime
date = datetime.datetime.now().strftime('%Y-%m-%d')
write_json_to_file(alertsummaries, f'data/labeling/alerts_{date}.json')

In [78]:
alertsummaries = read_data_from_json('data/labeling/alerts_2022-03-01.json')
alertsummaries2 = read_data_from_json('data/labeling/alerts_2021-09-13.json')

In [80]:
alert_ids = {alert['id'] for alert in alertsummaries}
alert_ids2 = {alert['id'] for alert in alertsummaries2}

In [81]:
len(alert_ids), len(alert_ids2), len(alert_ids.intersection(alert_ids2))

(4592, 4460, 2516)

In [82]:
for alert in alertsummaries2:
    if alert['id'] not in alert_ids:
        alertsummaries.append(alert)

In [40]:
selected_commits = get_selected_commits()
#hg_to_git, git_to_hg = get_hg_git_mapping()

Get selected commits: 583402it [02:37, 3705.89it/s] 


In [56]:
#labeling = 'fixed_defect_szz'

bugbug_szz_labeling = pd.read_csv(f'data/labeling/bugbug_szz.csv')
fixed_defect_szz_labeling = pd.read_csv(f'data/labeling/fixed_defect_szz.csv')
# fix_and_introducers = read_data_from_json(f'data/labeling/{labeling}/results/fix_and_introducers_pairs.json')

_, bugbug_fix_bug_ids_by_kind, _ = get_bugbug_regressors_and_fixes()

fixed_defect_fix_bug_ids_by_kind, _ = get_defects_and_fixes()


Get regressors and fixes: 205149it [00:47, 4357.69it/s]


Found
9673 regression
881 performance
1225 crash
281 security
32 memory
9 power
regressor bug ids,

and

15527 regression
976 performance
1595 crash
299 security
31 memory
9 power
fix bug ids.


Get defects and fixes: 205149it [02:13, 1534.85it/s]

Found
46122 regression
1304 security
963 performance
117 memory
3323 crash
39 power
fix bug ids.


In [67]:
bugbug_szz_labeling['performance'].sum()

317

In [68]:
fixed_defect_szz_labeling['performance'].sum()

567

In [69]:
import numpy as np
np.maximum(fixed_defect_szz_labeling['performance'], bugbug_szz_labeling['performance']).sum()

573

In [53]:
len(fix_bug_ids_by_kind['performance'])

963

In [33]:
import pandas as pd


In [37]:
fixes = set(fix for fix, introducer in fix_and_introducers)
len(fixes)

233

In [74]:
A = set(fixed_defect_fix_bug_ids_by_kind['performance'])
B = set(bugbug_fix_bug_ids_by_kind['performance'])
I = A.intersection(B)
len(I)

375

In [75]:
len(A.difference(I)), len(B.difference(I))

(588, 601)

In [83]:
alertsummaries_filtered = [a for a in alertsummaries if a['bug_number'] in fixed_defect_fix_bug_ids_by_kind['performance']]
len(alertsummaries_filtered)

304

In [84]:
alertsummaries_filtered = [a for a in alertsummaries if a['bug_number'] in bugbug_fix_bug_ids_by_kind['performance']]
len(alertsummaries_filtered)

549

In [65]:
alert_bug_ids = set(a['bug_number'] for a in alertsummaries)
len(alert_bug_ids.intersection(fixed_defect_fix_bug_ids_by_kind['performance']))

108

In [66]:
alert_bug_ids = set(a['bug_number'] for a in alertsummaries)
len(alert_bug_ids.intersection(bugbug_fix_bug_ids_by_kind['performance']))

258